# Load the pymoo result object saved with pickle.
ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

## Load ctaea results

In [2]:
with open('res_bc_Cogent_ctaea.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_ctaea.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_ctaea.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_ctaea.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_ctaea.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_ctaea.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_ctaea.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_ctaea.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_ctaea.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_ctaea.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_ctaea.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_ctaea.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [3]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_Cogent = F_Cogent[:, 2] != np.inf # FST should not be a infinite
c6_Cogent = F_Cogent[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c5_UsCarrier = F_UsCarrier[:, 2] != np.inf
c6_UsCarrier = F_UsCarrier[:, 3] != np.inf

c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c5_HiberniaGlobal = F_HiberniaGlobal[:, 2] != np.inf
c6_HiberniaGlobal = F_HiberniaGlobal[:, 3] != np.inf 

c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c5_Colt = F_Colt[:, 2] != np.inf
c6_Colt = F_Colt[:, 3] != np.inf 

c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c5_Funet = F_Funet[:, 2] != np.inf
c6_Funet = F_Funet[:, 3] != np.inf 

c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c5_Abvt = F_Abvt[:, 2] != np.inf
c6_Abvt = F_Abvt[:, 3] != np.inf 

c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c5_Intellifiber = F_Intellifiber[:, 2] != np.inf
c6_Intellifiber = F_Intellifiber[:, 3] != np.inf 

c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
c5_TataNld = F_TataNld[:, 2] != np.inf
c6_TataNld = F_TataNld[:, 3] != np.inf 

c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c5_Internode = F_Internode[:, 2] != np.inf
c6_Internode = F_Internode[:, 3] != np.inf 

c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c5_Missouri = F_Missouri[:, 2] != np.inf
c6_Missouri = F_Missouri[:, 3] != np.inf 

c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c5_Ion = F_Ion[:, 2] != np.inf
c6_Ion = F_Ion[:, 3] != np.inf 

c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 
c5_Palmetto = F_Palmetto[:, 2] != np.inf
c6_Palmetto = F_Palmetto[:, 3] != np.inf 

print('The following result is for ctaea:')

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c5_Cogent) and np.all(c6_Cogent):
    print("Cogent: " + 'True')
else: 
    print("Cogent: " + 'False')

if np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c5_UsCarrier) and np.all(c6_UsCarrier):
    print("UsCarrier: " + 'True')
else: 
    print("UsCarrier: " + 'False')

if np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c5_HiberniaGlobal) and np.all(c6_HiberniaGlobal):
    print("HiberniaGlobal: " + 'True')
else: 
    print("HiberniaGlobal: " + 'False')

if np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt) and np.all(c5_Colt) and np.all(c6_Colt):
    print("Colt: " + 'True')
else: 
    print("Colt: " + 'False')

if np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c5_Funet) and np.all(c6_Funet):
    print("Funet: " + 'True')
else: 
    print("Funet: " + 'False')

if np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c5_Abvt) and np.all(c6_Abvt):
    print("Abvt: " + 'True')
else: 
    print("Abvt: " + 'False')

if np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c5_Intellifiber) and np.all(c6_Intellifiber):
    print("Intellifiber: " + 'True')
else: 
    print("Intellifiber: " + 'False')

if np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c5_TataNld) and np.all(c6_TataNld):
    print("TataNld: " + 'True')
else: 
    print("TataNld: " + 'False')

if np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c5_Internode) and np.all(c6_Internode):
    print("Internode: " + 'True')
else: 
    print("Internode: " + 'False')

if np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c5_Missouri) and np.all(c6_Missouri):
    print("Missouri: " + 'True')
else: 
    print("Missouri: " + 'False')

if np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c5_Ion) and np.all(c6_Ion):
    print("Ion: " + 'True')
else: 
    print("Ion: " + 'False')

if np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto) and np.all(c5_Palmetto) and np.all(c6_Palmetto):
    print("Palmetto: " + 'True')
else: 
    print("Palmetto: " + 'False')

The following result is for ctaea:
Cogent: True
UsCarrier: True
HiberniaGlobal: True
Colt: True
Funet: True
Abvt: True
Intellifiber: True
TataNld: True
Internode: True
Missouri: True
Ion: True
Palmetto: True


## Load dnsga2 results

In [4]:
with open('res_bc_Cogent_dnsga2.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_dnsga2.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_dnsga2.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_dnsga2.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_dnsga2.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_dnsga2.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_dnsga2.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_dnsga2.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_dnsga2.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_dnsga2.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_dnsga2.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_dnsga2.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [5]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_Cogent = F_Cogent[:, 2] != np.inf # FST should not be a infinite
c6_Cogent = F_Cogent[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c5_UsCarrier = F_UsCarrier[:, 2] != np.inf
c6_UsCarrier = F_UsCarrier[:, 3] != np.inf

c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c5_HiberniaGlobal = F_HiberniaGlobal[:, 2] != np.inf
c6_HiberniaGlobal = F_HiberniaGlobal[:, 3] != np.inf 

c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c5_Colt = F_Colt[:, 2] != np.inf
c6_Colt = F_Colt[:, 3] != np.inf 

c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c5_Funet = F_Funet[:, 2] != np.inf
c6_Funet = F_Funet[:, 3] != np.inf 

c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c5_Abvt = F_Abvt[:, 2] != np.inf
c6_Abvt = F_Abvt[:, 3] != np.inf 

c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c5_Intellifiber = F_Intellifiber[:, 2] != np.inf
c6_Intellifiber = F_Intellifiber[:, 3] != np.inf 

c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
c5_TataNld = F_TataNld[:, 2] != np.inf
c6_TataNld = F_TataNld[:, 3] != np.inf 

c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c5_Internode = F_Internode[:, 2] != np.inf
c6_Internode = F_Internode[:, 3] != np.inf 

c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c5_Missouri = F_Missouri[:, 2] != np.inf
c6_Missouri = F_Missouri[:, 3] != np.inf 

c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c5_Ion = F_Ion[:, 2] != np.inf
c6_Ion = F_Ion[:, 3] != np.inf 

c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 
c5_Palmetto = F_Palmetto[:, 2] != np.inf
c6_Palmetto = F_Palmetto[:, 3] != np.inf 

print('The following result is for dnsga2:')

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c5_Cogent) and np.all(c6_Cogent):
    print("Cogent: " + 'True')
else: 
    print("Cogent: " + 'False')

if np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c5_UsCarrier) and np.all(c6_UsCarrier):
    print("UsCarrier: " + 'True')
else: 
    print("UsCarrier: " + 'False')

if np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c5_HiberniaGlobal) and np.all(c6_HiberniaGlobal):
    print("HiberniaGlobal: " + 'True')
else: 
    print("HiberniaGlobal: " + 'False')

if np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt) and np.all(c5_Colt) and np.all(c6_Colt):
    print("Colt: " + 'True')
else: 
    print("Colt: " + 'False')

if np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c5_Funet) and np.all(c6_Funet):
    print("Funet: " + 'True')
else: 
    print("Funet: " + 'False')

if np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c5_Abvt) and np.all(c6_Abvt):
    print("Abvt: " + 'True')
else: 
    print("Abvt: " + 'False')

if np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c5_Intellifiber) and np.all(c6_Intellifiber):
    print("Intellifiber: " + 'True')
else: 
    print("Intellifiber: " + 'False')

if np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c5_TataNld) and np.all(c6_TataNld):
    print("TataNld: " + 'True')
else: 
    print("TataNld: " + 'False')

if np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c5_Internode) and np.all(c6_Internode):
    print("Internode: " + 'True')
else: 
    print("Internode: " + 'False')

if np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c5_Missouri) and np.all(c6_Missouri):
    print("Missouri: " + 'True')
else: 
    print("Missouri: " + 'False')

if np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c5_Ion) and np.all(c6_Ion):
    print("Ion: " + 'True')
else: 
    print("Ion: " + 'False')

if np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto) and np.all(c5_Palmetto) and np.all(c6_Palmetto):
    print("Palmetto: " + 'True')
else: 
    print("Palmetto: " + 'False')

The following result is for dnsga2:
Cogent: False
UsCarrier: False
HiberniaGlobal: False
Colt: False
Funet: False
Abvt: False
Intellifiber: False
TataNld: False
Internode: False
Missouri: False
Ion: False
Palmetto: False


## Load kgbdmoea results

In [6]:
with open('res_bc_Cogent_kgbdmoea.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_kgbdmoea.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_kgbdmoea.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_kgbdmoea.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_kgbdmoea.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_kgbdmoea.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_kgbdmoea.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_kgbdmoea.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_kgbdmoea.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_kgbdmoea.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_kgbdmoea.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_kgbdmoea.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [7]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_Cogent = F_Cogent[:, 2] != np.inf # FST should not be a infinite
c6_Cogent = F_Cogent[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c5_UsCarrier = F_UsCarrier[:, 2] != np.inf
c6_UsCarrier = F_UsCarrier[:, 3] != np.inf

c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c5_HiberniaGlobal = F_HiberniaGlobal[:, 2] != np.inf
c6_HiberniaGlobal = F_HiberniaGlobal[:, 3] != np.inf 

c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c5_Colt = F_Colt[:, 2] != np.inf
c6_Colt = F_Colt[:, 3] != np.inf 

c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c5_Funet = F_Funet[:, 2] != np.inf
c6_Funet = F_Funet[:, 3] != np.inf 

c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c5_Abvt = F_Abvt[:, 2] != np.inf
c6_Abvt = F_Abvt[:, 3] != np.inf 

c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c5_Intellifiber = F_Intellifiber[:, 2] != np.inf
c6_Intellifiber = F_Intellifiber[:, 3] != np.inf 

c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
c5_TataNld = F_TataNld[:, 2] != np.inf
c6_TataNld = F_TataNld[:, 3] != np.inf 

c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c5_Internode = F_Internode[:, 2] != np.inf
c6_Internode = F_Internode[:, 3] != np.inf 

c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c5_Missouri = F_Missouri[:, 2] != np.inf
c6_Missouri = F_Missouri[:, 3] != np.inf 

c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c5_Ion = F_Ion[:, 2] != np.inf
c6_Ion = F_Ion[:, 3] != np.inf 

c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 
c5_Palmetto = F_Palmetto[:, 2] != np.inf
c6_Palmetto = F_Palmetto[:, 3] != np.inf 

print('The following result is for kgbdmoea:')

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c5_Cogent) and np.all(c6_Cogent):
    print("Cogent: " + 'True')
else: 
    print("Cogent: " + 'False')

if np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c5_UsCarrier) and np.all(c6_UsCarrier):
    print("UsCarrier: " + 'True')
else: 
    print("UsCarrier: " + 'False')

if np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c5_HiberniaGlobal) and np.all(c6_HiberniaGlobal):
    print("HiberniaGlobal: " + 'True')
else: 
    print("HiberniaGlobal: " + 'False')

if np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt) and np.all(c5_Colt) and np.all(c6_Colt):
    print("Colt: " + 'True')
else: 
    print("Colt: " + 'False')

if np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c5_Funet) and np.all(c6_Funet):
    print("Funet: " + 'True')
else: 
    print("Funet: " + 'False')

if np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c5_Abvt) and np.all(c6_Abvt):
    print("Abvt: " + 'True')
else: 
    print("Abvt: " + 'False')

if np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c5_Intellifiber) and np.all(c6_Intellifiber):
    print("Intellifiber: " + 'True')
else: 
    print("Intellifiber: " + 'False')

if np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c5_TataNld) and np.all(c6_TataNld):
    print("TataNld: " + 'True')
else: 
    print("TataNld: " + 'False')

if np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c5_Internode) and np.all(c6_Internode):
    print("Internode: " + 'True')
else: 
    print("Internode: " + 'False')

if np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c5_Missouri) and np.all(c6_Missouri):
    print("Missouri: " + 'True')
else: 
    print("Missouri: " + 'False')

if np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c5_Ion) and np.all(c6_Ion):
    print("Ion: " + 'True')
else: 
    print("Ion: " + 'False')

if np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto) and np.all(c5_Palmetto) and np.all(c6_Palmetto):
    print("Palmetto: " + 'True')
else: 
    print("Palmetto: " + 'False')

The following result is for kgbdmoea:
Cogent: False
UsCarrier: False
HiberniaGlobal: False
Colt: False
Funet: False
Abvt: False
Intellifiber: False
TataNld: False
Internode: False
Missouri: False
Ion: False
Palmetto: False


## Load moead results

In [8]:
with open('res_bc_Cogent_moead.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_moead.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_moead.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_moead.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_moead.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_moead.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_moead.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_moead.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_moead.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_moead.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_moead.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_moead.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [9]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_Cogent = F_Cogent[:, 2] != np.inf # FST should not be a infinite
c6_Cogent = F_Cogent[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c5_UsCarrier = F_UsCarrier[:, 2] != np.inf
c6_UsCarrier = F_UsCarrier[:, 3] != np.inf

c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c5_HiberniaGlobal = F_HiberniaGlobal[:, 2] != np.inf
c6_HiberniaGlobal = F_HiberniaGlobal[:, 3] != np.inf 

c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c5_Colt = F_Colt[:, 2] != np.inf
c6_Colt = F_Colt[:, 3] != np.inf 

c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c5_Funet = F_Funet[:, 2] != np.inf
c6_Funet = F_Funet[:, 3] != np.inf 

c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c5_Abvt = F_Abvt[:, 2] != np.inf
c6_Abvt = F_Abvt[:, 3] != np.inf 

c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c5_Intellifiber = F_Intellifiber[:, 2] != np.inf
c6_Intellifiber = F_Intellifiber[:, 3] != np.inf 

c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
c5_TataNld = F_TataNld[:, 2] != np.inf
c6_TataNld = F_TataNld[:, 3] != np.inf 

c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c5_Internode = F_Internode[:, 2] != np.inf
c6_Internode = F_Internode[:, 3] != np.inf 

c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c5_Missouri = F_Missouri[:, 2] != np.inf
c6_Missouri = F_Missouri[:, 3] != np.inf 

c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c5_Ion = F_Ion[:, 2] != np.inf
c6_Ion = F_Ion[:, 3] != np.inf 

c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 
c5_Palmetto = F_Palmetto[:, 2] != np.inf
c6_Palmetto = F_Palmetto[:, 3] != np.inf 

print('The following result is for moead:')

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c5_Cogent) and np.all(c6_Cogent):
    print("Cogent: " + 'True')
else: 
    print("Cogent: " + 'False')

if np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c5_UsCarrier) and np.all(c6_UsCarrier):
    print("UsCarrier: " + 'True')
else: 
    print("UsCarrier: " + 'False')

if np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c5_HiberniaGlobal) and np.all(c6_HiberniaGlobal):
    print("HiberniaGlobal: " + 'True')
else: 
    print("HiberniaGlobal: " + 'False')

if np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt) and np.all(c5_Colt) and np.all(c6_Colt):
    print("Colt: " + 'True')
else: 
    print("Colt: " + 'False')

if np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c5_Funet) and np.all(c6_Funet):
    print("Funet: " + 'True')
else: 
    print("Funet: " + 'False')

if np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c5_Abvt) and np.all(c6_Abvt):
    print("Abvt: " + 'True')
else: 
    print("Abvt: " + 'False')

if np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c5_Intellifiber) and np.all(c6_Intellifiber):
    print("Intellifiber: " + 'True')
else: 
    print("Intellifiber: " + 'False')

if np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c5_TataNld) and np.all(c6_TataNld):
    print("TataNld: " + 'True')
else: 
    print("TataNld: " + 'False')

if np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c5_Internode) and np.all(c6_Internode):
    print("Internode: " + 'True')
else: 
    print("Internode: " + 'False')

if np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c5_Missouri) and np.all(c6_Missouri):
    print("Missouri: " + 'True')
else: 
    print("Missouri: " + 'False')

if np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c5_Ion) and np.all(c6_Ion):
    print("Ion: " + 'True')
else: 
    print("Ion: " + 'False')

if np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto) and np.all(c5_Palmetto) and np.all(c6_Palmetto):
    print("Palmetto: " + 'True')
else: 
    print("Palmetto: " + 'False')

The following result is for moead:
Cogent: False
UsCarrier: False
HiberniaGlobal: False
Colt: False
Funet: False
Abvt: False
Intellifiber: False
TataNld: False
Internode: False
Missouri: False
Ion: False
Palmetto: False


## Load agemoea results

In [10]:
with open('res_bc_Cogent_agemoea.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_agemoea.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_agemoea.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_agemoea.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_agemoea.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_agemoea.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_agemoea.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_agemoea.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_agemoea.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_agemoea.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_agemoea.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_agemoea.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [11]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_Cogent = F_Cogent[:, 2] != np.inf # FST should not be a infinite
c6_Cogent = F_Cogent[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c5_UsCarrier = F_UsCarrier[:, 2] != np.inf
c6_UsCarrier = F_UsCarrier[:, 3] != np.inf

c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c5_HiberniaGlobal = F_HiberniaGlobal[:, 2] != np.inf
c6_HiberniaGlobal = F_HiberniaGlobal[:, 3] != np.inf 

c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c5_Colt = F_Colt[:, 2] != np.inf
c6_Colt = F_Colt[:, 3] != np.inf 

c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c5_Funet = F_Funet[:, 2] != np.inf
c6_Funet = F_Funet[:, 3] != np.inf 

c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c5_Abvt = F_Abvt[:, 2] != np.inf
c6_Abvt = F_Abvt[:, 3] != np.inf 

c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c5_Intellifiber = F_Intellifiber[:, 2] != np.inf
c6_Intellifiber = F_Intellifiber[:, 3] != np.inf 

c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
c5_TataNld = F_TataNld[:, 2] != np.inf
c6_TataNld = F_TataNld[:, 3] != np.inf 

c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c5_Internode = F_Internode[:, 2] != np.inf
c6_Internode = F_Internode[:, 3] != np.inf 

c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c5_Missouri = F_Missouri[:, 2] != np.inf
c6_Missouri = F_Missouri[:, 3] != np.inf 

c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c5_Ion = F_Ion[:, 2] != np.inf
c6_Ion = F_Ion[:, 3] != np.inf 

c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 
c5_Palmetto = F_Palmetto[:, 2] != np.inf
c6_Palmetto = F_Palmetto[:, 3] != np.inf 

print('The following result is for agemoea:')

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c5_Cogent) and np.all(c6_Cogent):
    print("Cogent: " + 'True')
else: 
    print("Cogent: " + 'False')

if np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c5_UsCarrier) and np.all(c6_UsCarrier):
    print("UsCarrier: " + 'True')
else: 
    print("UsCarrier: " + 'False')

if np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c5_HiberniaGlobal) and np.all(c6_HiberniaGlobal):
    print("HiberniaGlobal: " + 'True')
else: 
    print("HiberniaGlobal: " + 'False')

if np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt) and np.all(c5_Colt) and np.all(c6_Colt):
    print("Colt: " + 'True')
else: 
    print("Colt: " + 'False')

if np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c5_Funet) and np.all(c6_Funet):
    print("Funet: " + 'True')
else: 
    print("Funet: " + 'False')

if np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c5_Abvt) and np.all(c6_Abvt):
    print("Abvt: " + 'True')
else: 
    print("Abvt: " + 'False')

if np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c5_Intellifiber) and np.all(c6_Intellifiber):
    print("Intellifiber: " + 'True')
else: 
    print("Intellifiber: " + 'False')

if np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c5_TataNld) and np.all(c6_TataNld):
    print("TataNld: " + 'True')
else: 
    print("TataNld: " + 'False')

if np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c5_Internode) and np.all(c6_Internode):
    print("Internode: " + 'True')
else: 
    print("Internode: " + 'False')

if np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c5_Missouri) and np.all(c6_Missouri):
    print("Missouri: " + 'True')
else: 
    print("Missouri: " + 'False')

if np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c5_Ion) and np.all(c6_Ion):
    print("Ion: " + 'True')
else: 
    print("Ion: " + 'False')

if np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto) and np.all(c5_Palmetto) and np.all(c6_Palmetto):
    print("Palmetto: " + 'True')
else: 
    print("Palmetto: " + 'False')

The following result is for agemoea:
Cogent: True
UsCarrier: True
HiberniaGlobal: True
Colt: True
Funet: True
Abvt: True
Intellifiber: True
TataNld: True
Internode: True
Missouri: True
Ion: True
Palmetto: True


## Load agemoea2 results

In [12]:
with open('res_bc_Cogent_agemoea2.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_agemoea2.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_agemoea2.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_agemoea2.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_agemoea2.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_agemoea2.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_agemoea2.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_agemoea2.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_agemoea2.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_agemoea2.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_agemoea2.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_agemoea2.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [13]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_Cogent = F_Cogent[:, 2] != np.inf # FST should not be a infinite
c6_Cogent = F_Cogent[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c5_UsCarrier = F_UsCarrier[:, 2] != np.inf
c6_UsCarrier = F_UsCarrier[:, 3] != np.inf

c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c5_HiberniaGlobal = F_HiberniaGlobal[:, 2] != np.inf
c6_HiberniaGlobal = F_HiberniaGlobal[:, 3] != np.inf 

c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c5_Colt = F_Colt[:, 2] != np.inf
c6_Colt = F_Colt[:, 3] != np.inf 

c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c5_Funet = F_Funet[:, 2] != np.inf
c6_Funet = F_Funet[:, 3] != np.inf 

c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c5_Abvt = F_Abvt[:, 2] != np.inf
c6_Abvt = F_Abvt[:, 3] != np.inf 

c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c5_Intellifiber = F_Intellifiber[:, 2] != np.inf
c6_Intellifiber = F_Intellifiber[:, 3] != np.inf 

c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
c5_TataNld = F_TataNld[:, 2] != np.inf
c6_TataNld = F_TataNld[:, 3] != np.inf 

c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c5_Internode = F_Internode[:, 2] != np.inf
c6_Internode = F_Internode[:, 3] != np.inf 

c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c5_Missouri = F_Missouri[:, 2] != np.inf
c6_Missouri = F_Missouri[:, 3] != np.inf 

c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c5_Ion = F_Ion[:, 2] != np.inf
c6_Ion = F_Ion[:, 3] != np.inf 

c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 
c5_Palmetto = F_Palmetto[:, 2] != np.inf
c6_Palmetto = F_Palmetto[:, 3] != np.inf 

print('The following result is for agemoea2:')

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c5_Cogent) and np.all(c6_Cogent):
    print("Cogent: " + 'True')
else: 
    print("Cogent: " + 'False')

if np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c5_UsCarrier) and np.all(c6_UsCarrier):
    print("UsCarrier: " + 'True')
else: 
    print("UsCarrier: " + 'False')

if np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c5_HiberniaGlobal) and np.all(c6_HiberniaGlobal):
    print("HiberniaGlobal: " + 'True')
else: 
    print("HiberniaGlobal: " + 'False')

if np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt) and np.all(c5_Colt) and np.all(c6_Colt):
    print("Colt: " + 'True')
else: 
    print("Colt: " + 'False')

if np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c5_Funet) and np.all(c6_Funet):
    print("Funet: " + 'True')
else: 
    print("Funet: " + 'False')

if np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c5_Abvt) and np.all(c6_Abvt):
    print("Abvt: " + 'True')
else: 
    print("Abvt: " + 'False')

if np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c5_Intellifiber) and np.all(c6_Intellifiber):
    print("Intellifiber: " + 'True')
else: 
    print("Intellifiber: " + 'False')

if np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c5_TataNld) and np.all(c6_TataNld):
    print("TataNld: " + 'True')
else: 
    print("TataNld: " + 'False')

if np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c5_Internode) and np.all(c6_Internode):
    print("Internode: " + 'True')
else: 
    print("Internode: " + 'False')

if np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c5_Missouri) and np.all(c6_Missouri):
    print("Missouri: " + 'True')
else: 
    print("Missouri: " + 'False')

if np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c5_Ion) and np.all(c6_Ion):
    print("Ion: " + 'True')
else: 
    print("Ion: " + 'False')

if np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto) and np.all(c5_Palmetto) and np.all(c6_Palmetto):
    print("Palmetto: " + 'True')
else: 
    print("Palmetto: " + 'False')

The following result is for agemoea2:
Cogent: True
UsCarrier: True
HiberniaGlobal: True
Colt: True
Funet: True
Abvt: True
Intellifiber: True
TataNld: True
Internode: True
Missouri: True
Ion: True
Palmetto: True


## Load nsga2 results

In [14]:
with open('res_bc_Cogent_nsga2.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_nsga2.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_nsga2.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_nsga2.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_nsga2.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_nsga2.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_nsga2.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_nsga2.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_nsga2.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_nsga2.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_nsga2.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_nsga2.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [15]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_Cogent = F_Cogent[:, 2] != np.inf # FST should not be a infinite
c6_Cogent = F_Cogent[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c5_UsCarrier = F_UsCarrier[:, 2] != np.inf
c6_UsCarrier = F_UsCarrier[:, 3] != np.inf

c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c5_HiberniaGlobal = F_HiberniaGlobal[:, 2] != np.inf
c6_HiberniaGlobal = F_HiberniaGlobal[:, 3] != np.inf 

c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c5_Colt = F_Colt[:, 2] != np.inf
c6_Colt = F_Colt[:, 3] != np.inf 

c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c5_Funet = F_Funet[:, 2] != np.inf
c6_Funet = F_Funet[:, 3] != np.inf 

c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c5_Abvt = F_Abvt[:, 2] != np.inf
c6_Abvt = F_Abvt[:, 3] != np.inf 

c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c5_Intellifiber = F_Intellifiber[:, 2] != np.inf
c6_Intellifiber = F_Intellifiber[:, 3] != np.inf 

c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
c5_TataNld = F_TataNld[:, 2] != np.inf
c6_TataNld = F_TataNld[:, 3] != np.inf 

c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c5_Internode = F_Internode[:, 2] != np.inf
c6_Internode = F_Internode[:, 3] != np.inf 

c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c5_Missouri = F_Missouri[:, 2] != np.inf
c6_Missouri = F_Missouri[:, 3] != np.inf 

c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c5_Ion = F_Ion[:, 2] != np.inf
c6_Ion = F_Ion[:, 3] != np.inf 

c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 
c5_Palmetto = F_Palmetto[:, 2] != np.inf
c6_Palmetto = F_Palmetto[:, 3] != np.inf 

print('The following result is for nsga2:')

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c5_Cogent) and np.all(c6_Cogent):
    print("Cogent: " + 'True')
else: 
    print("Cogent: " + 'False')

if np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c5_UsCarrier) and np.all(c6_UsCarrier):
    print("UsCarrier: " + 'True')
else: 
    print("UsCarrier: " + 'False')

if np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c5_HiberniaGlobal) and np.all(c6_HiberniaGlobal):
    print("HiberniaGlobal: " + 'True')
else: 
    print("HiberniaGlobal: " + 'False')

if np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt) and np.all(c5_Colt) and np.all(c6_Colt):
    print("Colt: " + 'True')
else: 
    print("Colt: " + 'False')

if np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c5_Funet) and np.all(c6_Funet):
    print("Funet: " + 'True')
else: 
    print("Funet: " + 'False')

if np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c5_Abvt) and np.all(c6_Abvt):
    print("Abvt: " + 'True')
else: 
    print("Abvt: " + 'False')

if np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c5_Intellifiber) and np.all(c6_Intellifiber):
    print("Intellifiber: " + 'True')
else: 
    print("Intellifiber: " + 'False')

if np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c5_TataNld) and np.all(c6_TataNld):
    print("TataNld: " + 'True')
else: 
    print("TataNld: " + 'False')

if np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c5_Internode) and np.all(c6_Internode):
    print("Internode: " + 'True')
else: 
    print("Internode: " + 'False')

if np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c5_Missouri) and np.all(c6_Missouri):
    print("Missouri: " + 'True')
else: 
    print("Missouri: " + 'False')

if np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c5_Ion) and np.all(c6_Ion):
    print("Ion: " + 'True')
else: 
    print("Ion: " + 'False')

if np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto) and np.all(c5_Palmetto) and np.all(c6_Palmetto):
    print("Palmetto: " + 'True')
else: 
    print("Palmetto: " + 'False')

The following result is for nsga2:
Cogent: True
UsCarrier: True
HiberniaGlobal: True
Colt: True
Funet: True
Abvt: True
Intellifiber: True
TataNld: True
Internode: True
Missouri: True
Ion: True
Palmetto: True


## Load nsga3 results

In [16]:
with open('res_bc_Cogent_nsga3.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_nsga3.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_nsga3.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_nsga3.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_nsga3.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_nsga3.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_nsga3.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_nsga3.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_nsga3.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_nsga3.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_nsga3.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_nsga3.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [17]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_Cogent = F_Cogent[:, 2] != np.inf # FST should not be a infinite
c6_Cogent = F_Cogent[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c5_UsCarrier = F_UsCarrier[:, 2] != np.inf
c6_UsCarrier = F_UsCarrier[:, 3] != np.inf

c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c5_HiberniaGlobal = F_HiberniaGlobal[:, 2] != np.inf
c6_HiberniaGlobal = F_HiberniaGlobal[:, 3] != np.inf 

c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c5_Colt = F_Colt[:, 2] != np.inf
c6_Colt = F_Colt[:, 3] != np.inf 

c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c5_Funet = F_Funet[:, 2] != np.inf
c6_Funet = F_Funet[:, 3] != np.inf 

c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c5_Abvt = F_Abvt[:, 2] != np.inf
c6_Abvt = F_Abvt[:, 3] != np.inf 

c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c5_Intellifiber = F_Intellifiber[:, 2] != np.inf
c6_Intellifiber = F_Intellifiber[:, 3] != np.inf 

c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
c5_TataNld = F_TataNld[:, 2] != np.inf
c6_TataNld = F_TataNld[:, 3] != np.inf 

c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c5_Internode = F_Internode[:, 2] != np.inf
c6_Internode = F_Internode[:, 3] != np.inf 

c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c5_Missouri = F_Missouri[:, 2] != np.inf
c6_Missouri = F_Missouri[:, 3] != np.inf 

c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c5_Ion = F_Ion[:, 2] != np.inf
c6_Ion = F_Ion[:, 3] != np.inf 

c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 
c5_Palmetto = F_Palmetto[:, 2] != np.inf
c6_Palmetto = F_Palmetto[:, 3] != np.inf 

print('The following result is for nsga3:')

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c5_Cogent) and np.all(c6_Cogent):
    print("Cogent: " + 'True')
else: 
    print("Cogent: " + 'False')

if np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c5_UsCarrier) and np.all(c6_UsCarrier):
    print("UsCarrier: " + 'True')
else: 
    print("UsCarrier: " + 'False')

if np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c5_HiberniaGlobal) and np.all(c6_HiberniaGlobal):
    print("HiberniaGlobal: " + 'True')
else: 
    print("HiberniaGlobal: " + 'False')

if np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt) and np.all(c5_Colt) and np.all(c6_Colt):
    print("Colt: " + 'True')
else: 
    print("Colt: " + 'False')

if np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c5_Funet) and np.all(c6_Funet):
    print("Funet: " + 'True')
else: 
    print("Funet: " + 'False')

if np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c5_Abvt) and np.all(c6_Abvt):
    print("Abvt: " + 'True')
else: 
    print("Abvt: " + 'False')

if np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c5_Intellifiber) and np.all(c6_Intellifiber):
    print("Intellifiber: " + 'True')
else: 
    print("Intellifiber: " + 'False')

if np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c5_TataNld) and np.all(c6_TataNld):
    print("TataNld: " + 'True')
else: 
    print("TataNld: " + 'False')

if np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c5_Internode) and np.all(c6_Internode):
    print("Internode: " + 'True')
else: 
    print("Internode: " + 'False')

if np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c5_Missouri) and np.all(c6_Missouri):
    print("Missouri: " + 'True')
else: 
    print("Missouri: " + 'False')

if np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c5_Ion) and np.all(c6_Ion):
    print("Ion: " + 'True')
else: 
    print("Ion: " + 'False')

if np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto) and np.all(c5_Palmetto) and np.all(c6_Palmetto):
    print("Palmetto: " + 'True')
else: 
    print("Palmetto: " + 'False')

The following result is for nsga3:
Cogent: True
UsCarrier: True
HiberniaGlobal: True
Colt: True
Funet: True
Abvt: True
Intellifiber: True
TataNld: True
Internode: True
Missouri: True
Ion: True
Palmetto: True


## Load rnsga2 results

In [18]:
with open('res_bc_Cogent_rnsga2.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_rnsga2.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_rnsga2.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_rnsga2.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_rnsga2.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_rnsga2.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_rnsga2.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_rnsga2.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_rnsga2.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_rnsga2.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_rnsga2.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_rnsga2.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [19]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_Cogent = F_Cogent[:, 2] != np.inf # FST should not be a infinite
c6_Cogent = F_Cogent[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c5_UsCarrier = F_UsCarrier[:, 2] != np.inf
c6_UsCarrier = F_UsCarrier[:, 3] != np.inf

c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c5_HiberniaGlobal = F_HiberniaGlobal[:, 2] != np.inf
c6_HiberniaGlobal = F_HiberniaGlobal[:, 3] != np.inf 

c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c5_Colt = F_Colt[:, 2] != np.inf
c6_Colt = F_Colt[:, 3] != np.inf 

c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c5_Funet = F_Funet[:, 2] != np.inf
c6_Funet = F_Funet[:, 3] != np.inf 

c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c5_Abvt = F_Abvt[:, 2] != np.inf
c6_Abvt = F_Abvt[:, 3] != np.inf 

c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c5_Intellifiber = F_Intellifiber[:, 2] != np.inf
c6_Intellifiber = F_Intellifiber[:, 3] != np.inf 

c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
c5_TataNld = F_TataNld[:, 2] != np.inf
c6_TataNld = F_TataNld[:, 3] != np.inf 

c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c5_Internode = F_Internode[:, 2] != np.inf
c6_Internode = F_Internode[:, 3] != np.inf 

c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c5_Missouri = F_Missouri[:, 2] != np.inf
c6_Missouri = F_Missouri[:, 3] != np.inf 

c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c5_Ion = F_Ion[:, 2] != np.inf
c6_Ion = F_Ion[:, 3] != np.inf 

c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 
c5_Palmetto = F_Palmetto[:, 2] != np.inf
c6_Palmetto = F_Palmetto[:, 3] != np.inf 

print('The following result is for rnsga2:')

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c5_Cogent) and np.all(c6_Cogent):
    print("Cogent: " + 'True')
else: 
    print("Cogent: " + 'False')

if np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c5_UsCarrier) and np.all(c6_UsCarrier):
    print("UsCarrier: " + 'True')
else: 
    print("UsCarrier: " + 'False')

if np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c5_HiberniaGlobal) and np.all(c6_HiberniaGlobal):
    print("HiberniaGlobal: " + 'True')
else: 
    print("HiberniaGlobal: " + 'False')

if np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt) and np.all(c5_Colt) and np.all(c6_Colt):
    print("Colt: " + 'True')
else: 
    print("Colt: " + 'False')

if np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c5_Funet) and np.all(c6_Funet):
    print("Funet: " + 'True')
else: 
    print("Funet: " + 'False')

if np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c5_Abvt) and np.all(c6_Abvt):
    print("Abvt: " + 'True')
else: 
    print("Abvt: " + 'False')

if np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c5_Intellifiber) and np.all(c6_Intellifiber):
    print("Intellifiber: " + 'True')
else: 
    print("Intellifiber: " + 'False')

if np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c5_TataNld) and np.all(c6_TataNld):
    print("TataNld: " + 'True')
else: 
    print("TataNld: " + 'False')

if np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c5_Internode) and np.all(c6_Internode):
    print("Internode: " + 'True')
else: 
    print("Internode: " + 'False')

if np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c5_Missouri) and np.all(c6_Missouri):
    print("Missouri: " + 'True')
else: 
    print("Missouri: " + 'False')

if np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c5_Ion) and np.all(c6_Ion):
    print("Ion: " + 'True')
else: 
    print("Ion: " + 'False')

if np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto) and np.all(c5_Palmetto) and np.all(c6_Palmetto):
    print("Palmetto: " + 'True')
else: 
    print("Palmetto: " + 'False')

The following result is for rnsga2:
Cogent: True
UsCarrier: True
HiberniaGlobal: True
Colt: True
Funet: True
Abvt: True
Intellifiber: True
TataNld: True
Internode: True
Missouri: True
Ion: True
Palmetto: True


## Load rnsga3 results

In [20]:
with open('res_bc_Cogent_rnsga3.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_rnsga3.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_rnsga3.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_rnsga3.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_rnsga3.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_rnsga3.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_rnsga3.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_rnsga3.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_rnsga3.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_rnsga3.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_rnsga3.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_rnsga3.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [21]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_Cogent = F_Cogent[:, 2] != np.inf # FST should not be a infinite
c6_Cogent = F_Cogent[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c5_UsCarrier = F_UsCarrier[:, 2] != np.inf
c6_UsCarrier = F_UsCarrier[:, 3] != np.inf

c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c5_HiberniaGlobal = F_HiberniaGlobal[:, 2] != np.inf
c6_HiberniaGlobal = F_HiberniaGlobal[:, 3] != np.inf 

c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c5_Colt = F_Colt[:, 2] != np.inf
c6_Colt = F_Colt[:, 3] != np.inf 

c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c5_Funet = F_Funet[:, 2] != np.inf
c6_Funet = F_Funet[:, 3] != np.inf 

c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c5_Abvt = F_Abvt[:, 2] != np.inf
c6_Abvt = F_Abvt[:, 3] != np.inf 

c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c5_Intellifiber = F_Intellifiber[:, 2] != np.inf
c6_Intellifiber = F_Intellifiber[:, 3] != np.inf 

c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
c5_TataNld = F_TataNld[:, 2] != np.inf
c6_TataNld = F_TataNld[:, 3] != np.inf 

c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c5_Internode = F_Internode[:, 2] != np.inf
c6_Internode = F_Internode[:, 3] != np.inf 

c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c5_Missouri = F_Missouri[:, 2] != np.inf
c6_Missouri = F_Missouri[:, 3] != np.inf 

c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c5_Ion = F_Ion[:, 2] != np.inf
c6_Ion = F_Ion[:, 3] != np.inf 

c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 
c5_Palmetto = F_Palmetto[:, 2] != np.inf
c6_Palmetto = F_Palmetto[:, 3] != np.inf 

print('The following result is for rnsga3:')

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c5_Cogent) and np.all(c6_Cogent):
    print("Cogent: " + 'True')
else: 
    print("Cogent: " + 'False')

if np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c5_UsCarrier) and np.all(c6_UsCarrier):
    print("UsCarrier: " + 'True')
else: 
    print("UsCarrier: " + 'False')

if np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c5_HiberniaGlobal) and np.all(c6_HiberniaGlobal):
    print("HiberniaGlobal: " + 'True')
else: 
    print("HiberniaGlobal: " + 'False')

if np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt) and np.all(c5_Colt) and np.all(c6_Colt):
    print("Colt: " + 'True')
else: 
    print("Colt: " + 'False')

if np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c5_Funet) and np.all(c6_Funet):
    print("Funet: " + 'True')
else: 
    print("Funet: " + 'False')

if np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c5_Abvt) and np.all(c6_Abvt):
    print("Abvt: " + 'True')
else: 
    print("Abvt: " + 'False')

if np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c5_Intellifiber) and np.all(c6_Intellifiber):
    print("Intellifiber: " + 'True')
else: 
    print("Intellifiber: " + 'False')

if np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c5_TataNld) and np.all(c6_TataNld):
    print("TataNld: " + 'True')
else: 
    print("TataNld: " + 'False')

if np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c5_Internode) and np.all(c6_Internode):
    print("Internode: " + 'True')
else: 
    print("Internode: " + 'False')

if np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c5_Missouri) and np.all(c6_Missouri):
    print("Missouri: " + 'True')
else: 
    print("Missouri: " + 'False')

if np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c5_Ion) and np.all(c6_Ion):
    print("Ion: " + 'True')
else: 
    print("Ion: " + 'False')

if np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto) and np.all(c5_Palmetto) and np.all(c6_Palmetto):
    print("Palmetto: " + 'True')
else: 
    print("Palmetto: " + 'False')

The following result is for rnsga3:
Cogent: True
UsCarrier: True
HiberniaGlobal: True
Colt: True
Funet: True
Abvt: True
Intellifiber: True
TataNld: True
Internode: True
Missouri: True
Ion: True
Palmetto: True


## Load rvea results

In [22]:
with open('res_bc_Cogent_rvea.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_rvea.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_rvea.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_rvea.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_rvea.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_rvea.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_rvea.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_rvea.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_rvea.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_rvea.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_rvea.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_rvea.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [23]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_Cogent = F_Cogent[:, 2] != np.inf # FST should not be a infinite
c6_Cogent = F_Cogent[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c5_UsCarrier = F_UsCarrier[:, 2] != np.inf
c6_UsCarrier = F_UsCarrier[:, 3] != np.inf

c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c5_HiberniaGlobal = F_HiberniaGlobal[:, 2] != np.inf
c6_HiberniaGlobal = F_HiberniaGlobal[:, 3] != np.inf 

c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c5_Colt = F_Colt[:, 2] != np.inf
c6_Colt = F_Colt[:, 3] != np.inf 

c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c5_Funet = F_Funet[:, 2] != np.inf
c6_Funet = F_Funet[:, 3] != np.inf 

c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c5_Abvt = F_Abvt[:, 2] != np.inf
c6_Abvt = F_Abvt[:, 3] != np.inf 

c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c5_Intellifiber = F_Intellifiber[:, 2] != np.inf
c6_Intellifiber = F_Intellifiber[:, 3] != np.inf 

c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
c5_TataNld = F_TataNld[:, 2] != np.inf
c6_TataNld = F_TataNld[:, 3] != np.inf 

c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c5_Internode = F_Internode[:, 2] != np.inf
c6_Internode = F_Internode[:, 3] != np.inf 

c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c5_Missouri = F_Missouri[:, 2] != np.inf
c6_Missouri = F_Missouri[:, 3] != np.inf 

c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c5_Ion = F_Ion[:, 2] != np.inf
c6_Ion = F_Ion[:, 3] != np.inf 

c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 
c5_Palmetto = F_Palmetto[:, 2] != np.inf
c6_Palmetto = F_Palmetto[:, 3] != np.inf 

print('The following result is for rvea:')

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c5_Cogent) and np.all(c6_Cogent):
    print("Cogent: " + 'True')
else: 
    print("Cogent: " + 'False')

if np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c5_UsCarrier) and np.all(c6_UsCarrier):
    print("UsCarrier: " + 'True')
else: 
    print("UsCarrier: " + 'False')

if np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c5_HiberniaGlobal) and np.all(c6_HiberniaGlobal):
    print("HiberniaGlobal: " + 'True')
else: 
    print("HiberniaGlobal: " + 'False')

if np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt) and np.all(c5_Colt) and np.all(c6_Colt):
    print("Colt: " + 'True')
else: 
    print("Colt: " + 'False')

if np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c5_Funet) and np.all(c6_Funet):
    print("Funet: " + 'True')
else: 
    print("Funet: " + 'False')

if np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c5_Abvt) and np.all(c6_Abvt):
    print("Abvt: " + 'True')
else: 
    print("Abvt: " + 'False')

if np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c5_Intellifiber) and np.all(c6_Intellifiber):
    print("Intellifiber: " + 'True')
else: 
    print("Intellifiber: " + 'False')

if np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c5_TataNld) and np.all(c6_TataNld):
    print("TataNld: " + 'True')
else: 
    print("TataNld: " + 'False')

if np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c5_Internode) and np.all(c6_Internode):
    print("Internode: " + 'True')
else: 
    print("Internode: " + 'False')

if np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c5_Missouri) and np.all(c6_Missouri):
    print("Missouri: " + 'True')
else: 
    print("Missouri: " + 'False')

if np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c5_Ion) and np.all(c6_Ion):
    print("Ion: " + 'True')
else: 
    print("Ion: " + 'False')

if np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto) and np.all(c5_Palmetto) and np.all(c6_Palmetto):
    print("Palmetto: " + 'True')
else: 
    print("Palmetto: " + 'False')

The following result is for rvea:
Cogent: False
UsCarrier: False
HiberniaGlobal: False
Colt: False
Funet: False
Abvt: False
Intellifiber: False
TataNld: False
Internode: False
Missouri: False
Ion: False
Palmetto: False


## Load smsemoa results

In [24]:
with open('res_bc_Cogent_smsemoa.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_smsemoa.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_smsemoa.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_smsemoa.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_smsemoa.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_smsemoa.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_smsemoa.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_smsemoa.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_smsemoa.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_smsemoa.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_smsemoa.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_smsemoa.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [25]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_Cogent = F_Cogent[:, 2] != np.inf # FST should not be a infinite
c6_Cogent = F_Cogent[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c5_UsCarrier = F_UsCarrier[:, 2] != np.inf
c6_UsCarrier = F_UsCarrier[:, 3] != np.inf

c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c5_HiberniaGlobal = F_HiberniaGlobal[:, 2] != np.inf
c6_HiberniaGlobal = F_HiberniaGlobal[:, 3] != np.inf 

c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c5_Colt = F_Colt[:, 2] != np.inf
c6_Colt = F_Colt[:, 3] != np.inf 

c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c5_Funet = F_Funet[:, 2] != np.inf
c6_Funet = F_Funet[:, 3] != np.inf 

c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c5_Abvt = F_Abvt[:, 2] != np.inf
c6_Abvt = F_Abvt[:, 3] != np.inf 

c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c5_Intellifiber = F_Intellifiber[:, 2] != np.inf
c6_Intellifiber = F_Intellifiber[:, 3] != np.inf 

c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
c5_TataNld = F_TataNld[:, 2] != np.inf
c6_TataNld = F_TataNld[:, 3] != np.inf 

c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c5_Internode = F_Internode[:, 2] != np.inf
c6_Internode = F_Internode[:, 3] != np.inf 

c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c5_Missouri = F_Missouri[:, 2] != np.inf
c6_Missouri = F_Missouri[:, 3] != np.inf 

c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c5_Ion = F_Ion[:, 2] != np.inf
c6_Ion = F_Ion[:, 3] != np.inf 

c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 
c5_Palmetto = F_Palmetto[:, 2] != np.inf
c6_Palmetto = F_Palmetto[:, 3] != np.inf 

print('The following result is for smsemoa:')

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c5_Cogent) and np.all(c6_Cogent):
    print("Cogent: " + 'True')
else: 
    print("Cogent: " + 'False')

if np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c5_UsCarrier) and np.all(c6_UsCarrier):
    print("UsCarrier: " + 'True')
else: 
    print("UsCarrier: " + 'False')

if np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c5_HiberniaGlobal) and np.all(c6_HiberniaGlobal):
    print("HiberniaGlobal: " + 'True')
else: 
    print("HiberniaGlobal: " + 'False')

if np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt) and np.all(c5_Colt) and np.all(c6_Colt):
    print("Colt: " + 'True')
else: 
    print("Colt: " + 'False')

if np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c5_Funet) and np.all(c6_Funet):
    print("Funet: " + 'True')
else: 
    print("Funet: " + 'False')

if np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c5_Abvt) and np.all(c6_Abvt):
    print("Abvt: " + 'True')
else: 
    print("Abvt: " + 'False')

if np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c5_Intellifiber) and np.all(c6_Intellifiber):
    print("Intellifiber: " + 'True')
else: 
    print("Intellifiber: " + 'False')

if np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c5_TataNld) and np.all(c6_TataNld):
    print("TataNld: " + 'True')
else: 
    print("TataNld: " + 'False')

if np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c5_Internode) and np.all(c6_Internode):
    print("Internode: " + 'True')
else: 
    print("Internode: " + 'False')

if np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c5_Missouri) and np.all(c6_Missouri):
    print("Missouri: " + 'True')
else: 
    print("Missouri: " + 'False')

if np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c5_Ion) and np.all(c6_Ion):
    print("Ion: " + 'True')
else: 
    print("Ion: " + 'False')

if np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto) and np.all(c5_Palmetto) and np.all(c6_Palmetto):
    print("Palmetto: " + 'True')
else: 
    print("Palmetto: " + 'False')

The following result is for smsemoa:
Cogent: True
UsCarrier: True
HiberniaGlobal: True
Colt: True
Funet: True
Abvt: True
Intellifiber: True
TataNld: True
Internode: True
Missouri: True
Ion: True
Palmetto: True


## Load unsga3 results

In [26]:
with open('res_bc_Cogent_unsga3.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_unsga3.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_unsga3.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_unsga3.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_unsga3.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_unsga3.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_unsga3.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_unsga3.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_unsga3.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_unsga3.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_unsga3.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_unsga3.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [27]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c5_Cogent = F_Cogent[:, 2] != np.inf # FST should not be a infinite
c6_Cogent = F_Cogent[:, 3] != np.inf # Variance of controller's betweenness centrality should not be a infinite

c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c5_UsCarrier = F_UsCarrier[:, 2] != np.inf
c6_UsCarrier = F_UsCarrier[:, 3] != np.inf

c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c5_HiberniaGlobal = F_HiberniaGlobal[:, 2] != np.inf
c6_HiberniaGlobal = F_HiberniaGlobal[:, 3] != np.inf 

c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c5_Colt = F_Colt[:, 2] != np.inf
c6_Colt = F_Colt[:, 3] != np.inf 

c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c5_Funet = F_Funet[:, 2] != np.inf
c6_Funet = F_Funet[:, 3] != np.inf 

c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c5_Abvt = F_Abvt[:, 2] != np.inf
c6_Abvt = F_Abvt[:, 3] != np.inf 

c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c5_Intellifiber = F_Intellifiber[:, 2] != np.inf
c6_Intellifiber = F_Intellifiber[:, 3] != np.inf 

c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
c5_TataNld = F_TataNld[:, 2] != np.inf
c6_TataNld = F_TataNld[:, 3] != np.inf 

c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c5_Internode = F_Internode[:, 2] != np.inf
c6_Internode = F_Internode[:, 3] != np.inf 

c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c5_Missouri = F_Missouri[:, 2] != np.inf
c6_Missouri = F_Missouri[:, 3] != np.inf 

c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c5_Ion = F_Ion[:, 2] != np.inf
c6_Ion = F_Ion[:, 3] != np.inf 

c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 
c5_Palmetto = F_Palmetto[:, 2] != np.inf
c6_Palmetto = F_Palmetto[:, 3] != np.inf 

print('The following result is for unsga3:')

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c5_Cogent) and np.all(c6_Cogent):
    print("Cogent: " + 'True')
else: 
    print("Cogent: " + 'False')

if np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c5_UsCarrier) and np.all(c6_UsCarrier):
    print("UsCarrier: " + 'True')
else: 
    print("UsCarrier: " + 'False')

if np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c5_HiberniaGlobal) and np.all(c6_HiberniaGlobal):
    print("HiberniaGlobal: " + 'True')
else: 
    print("HiberniaGlobal: " + 'False')

if np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt) and np.all(c5_Colt) and np.all(c6_Colt):
    print("Colt: " + 'True')
else: 
    print("Colt: " + 'False')

if np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c5_Funet) and np.all(c6_Funet):
    print("Funet: " + 'True')
else: 
    print("Funet: " + 'False')

if np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c5_Abvt) and np.all(c6_Abvt):
    print("Abvt: " + 'True')
else: 
    print("Abvt: " + 'False')

if np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c5_Intellifiber) and np.all(c6_Intellifiber):
    print("Intellifiber: " + 'True')
else: 
    print("Intellifiber: " + 'False')

if np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c5_TataNld) and np.all(c6_TataNld):
    print("TataNld: " + 'True')
else: 
    print("TataNld: " + 'False')

if np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c5_Internode) and np.all(c6_Internode):
    print("Internode: " + 'True')
else: 
    print("Internode: " + 'False')

if np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c5_Missouri) and np.all(c6_Missouri):
    print("Missouri: " + 'True')
else: 
    print("Missouri: " + 'False')

if np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c5_Ion) and np.all(c6_Ion):
    print("Ion: " + 'True')
else: 
    print("Ion: " + 'False')

if np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto) and np.all(c5_Palmetto) and np.all(c6_Palmetto):
    print("Palmetto: " + 'True')
else: 
    print("Palmetto: " + 'False')

The following result is for unsga3:
Cogent: True
UsCarrier: True
HiberniaGlobal: True
Colt: True
Funet: True
Abvt: True
Intellifiber: True
TataNld: True
Internode: True
Missouri: True
Ion: True
Palmetto: True
